# Select records from a DataFrame by the n largest values per group

Based on answers to the following stackoverflow posts:
- https://stackoverflow.com/questions/63431597/how-to-drop-certain-values-within-a-multi-level-index-python-pandas/63432079#63432079
- https://stackoverflow.com/questions/27842613/pandas-groupby-sort-within-groups

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Data is from the Stackoverflow question linked above

data = {
    'full_name':
        ['Michael Thomas', 'Chris Godwin', 'DeAndre Hopkins', 'Julio Jones', 'Cooper Kupp', 'Julian Edelman', 
         'Kenny Golladay', 'Keenan Allen', 'Amari Cooper', 'D.J. Moore', 'Mike Evans', 'DeVante Parker', 
         'Jarvis Landry', 'Stefon Diggs', 'Tyler Lockett', 'Tyler Boyd', 'John Brown', 'Robert Woods', 
         'Courtland Sutton', 'Calvin Ridley', 'A.J. Brown', 'Terry McLaurin', 'Davante Adams', 'Cole Beasley', 
         'Michael Gallup', 'Tyreek Hill', 'Jamison Crowder', 'Larry Fitzgerald', 'Curtis Samuel', 'Deebo Samuel', 
         'Darius Slayton', 'Mike Williams', 'Mike Williams', 'Robby Anderson', 'Christian Kirk', 
         'Diontae Johnson', 'Chris Conley', 'Randall Cobb', 'Tyrell Williams', 'Marquise Brown', 'Sammy Watkins', 
         'Golden Tate', 'James Washington', 'Dede Westbrook', 'Danny Amendola', 'Sterling Shepard', 
         'Zach Pascal', 'Anthony Miller', 'Alshon Jeffery', 'Kenny Stills', 'T.Y. Hilton', 'Adam Thielen', 
         'Breshad Perriman', 'Mecole Hardman', 'JuJu Smith-Schuster', 'Hunter Renfrow', 'Brandin Cooks', 
         'Corey Davis', 'Auden Tate', 'Emmanuel Sanders', 'Emmanuel Sanders', 'Alex Erickson', 'Kendrick Bourne', 
         'Nelson Agholor', 'Preston Williams', 'Demarcus Robinson', 'Taylor Gabriel', 'Russell Gage', 
         'Adam Humphries', 'Allen Lazard', 'Allen Hurns', 'Demaryius Thomas', 'Marquez Valdes-Scantling', 
         'Albert Wilson', 'Bisi Johnson', 'Geronimo Allison', 'Paul Richardson', 'Keelan Cole', 'Cody Latimer', 
         'Jakobi Meyers', 'Josh Reynolds', 'Kelvin Harmon', 'Seth Roberts', 'Isaiah McKenzie', 'David Moore', 
         'Josh Gordon', 'Josh Gordon', "Tre'Quan Smith", 'Jarius Wright', 'Damiere Byrd', 'Keke Coutee', 
         'DaeSean Hamilton', 'Pharoh Cooper', 'Trey Quinn', 'Miles Boykin', 'Jaron Brown', 'Chester Rogers', 
         'Tavon Austin', 'KeeSean Johnson', 'Malik Turner', 'Bennie Fowler', 'Vyncint Smith', 'Jakeem Grant', 
         'Parris Campbell', 'Marvin Hall', 'Jake Kumerow', 'Justin Watson', 'Marquise Goodwin', 'Javon Wims', 
         'DeSean Jackson', 'Andy Isabella', 'Tim Patrick', 'Byron Pringle', 'Dante Pettis', 'Laquon Treadwell', 
         'J.J. Arcega-Whiteside', "N'Keal Harry", 'Justin Hardy', 'Kalif Raymond', 'Zay Jones', 'Zay Jones', 
         'Trevor Davis', 'Trevor Davis', 'Trevor Davis', 'Cordarrelle Patterson', 'Keelan Doss', 'Damion Ratley', 
         'Mack Hollins', 'Mack Hollins', 'Devin Smith', 'Dontrelle Inman', 'Dontrelle Inman', 'Christian Blake', 
         'Duke Williams', 'Antonio Callaway', 'Olamide Zaccheaus', 'Rashard Higgins', 'Braxton Berrios', 
         'DeAndre Carter', 'Robert Foster', 'Deon Cain', 'Deon Cain', 'Trent Sherfield', 'Andre Patton', 
         'Chris Hogan', 'Ryan Switzer', 'Diontae Spencer', 'Deonte Harris', 'Chad Beebe', 'Marcell Ateman', 
         'Travis Benjamin', 'KhaDarel Hodge', 'Ventell Bryant', 'Geremy Davis', 'Jason Moore', 'Devin Funchess', 
         'Johnny Holton', 'Cody Core', 'Donte Moncrief', 'Andre Roberts', 'Russell Shepard', 'Gunner Olszewski', 
         'Ryan Grant', 'C.J. Board', 'Chris Moore', 'Marqise Lee', 'Stanley Morgan', 'Riley Ridley', 
         'Fred Brown', 'DeAndrew White', 'Josh Bellamy', 'Ashton Dulin', 'Michael Walker', 'Mike Thomas', 
         'Brandon Zylstra', 'Austin Carr', 'Dwayne Harris', 'Krishawn Hogan', 'Quincy Enunwa', 'Greg Dortch', 
         'JoJo Natson', 'Juwann Winfree', 'Matthew Slater', 'Taywan Taylor'],
    'rec_yards':
        [1688, 1333, 1165, 1316, 1062, 1091, 1118, 1117, 1097, 1175, 1157, 1065, 1092, 1130, 1006, 987, 1060, 
         1067, 1060, 866, 927, 919, 904, 778, 1009, 799, 767, 759, 614, 700, 690, 963, 963, 761, 649, 626, 737, 
         747, 651, 569, 665, 608, 735, 588, 662, 537, 597, 651, 490, 561, 429, 418, 511, 508, 546, 503, 543, 
         557, 575, 477, 367, 513, 358, 363, 428, 425, 353, 378, 374, 408, 416, 433, 433, 292, 260, 270, 245, 
         294, 288, 359, 326, 332, 271, 247, 271, 287, 139, 178, 286, 285, 247, 232, 219, 198, 198, 220, 179, 
         176, 187, 245, 193, 189, 164, 127, 261, 212, 132, 186, 163, 159, 189, 204, 170, 109, 184, 169, 76, 
         155, 170, 126, 69, 83, 28, 0, 83, 133, 136, 125, 0, 113, 132, 43, 91, 58, 89, 93, 55, 104, 97, 64, 72, 
         52, 80, 56, 53, 27, 31, 24, 70, 70, 30, 57, 15, 38, 43, 32, 21, 28, 18, 20, 25, 34, 14, 31, 21, 18, 
         18, 15, 21, 20, 20, 17, 15, 14, 10, 9, 7, 4, -4, 0, 0, 0, 0, 0],
    'team':
        ['NO', 'TB', 'HOU', 'ATL', 'LA', 'NE', 'DET', 'LAC', 'DAL', 'CAR', 'TB', 'MIA', 'CLE', 'MIN', 'SEA', 
         'CIN', 'BUF', 'LA', 'DEN', 'ATL', 'TEN', 'WAS', 'GB', 'BUF', 'DAL', 'KC', 'NYJ', 'ARI', 'CAR', 'SF', 
         'NYG', 'LAC', 'LAC', 'NYJ', 'ARI', 'PIT', 'JAC', 'DAL', 'OAK', 'BAL', 'KC', 'NYG', 'PIT', 'JAC', 
         'DET', 'NYG', 'IND', 'CHI', 'PHI', 'HOU', 'IND', 'MIN', 'TB', 'KC', 'PIT', 'OAK', 'LA', 'TEN', 'CIN', 
         'SF', 'DEN', 'CIN', 'SF', 'PHI', 'MIA', 'KC', 'CHI', 'ATL', 'TEN', 'GB', 'MIA', 'NYJ', 'GB', 'MIA', 
         'MIN', 'GB', 'WAS', 'JAC', 'NYG', 'NE', 'LA', 'WAS', 'BAL', 'BUF', 'SEA', 'NE', 'SEA', 'NO', 'CAR', 
         'ARI', 'HOU', 'DEN', 'ARI', 'WAS', 'BAL', 'SEA', 'IND', 'DAL', 'ARI', 'SEA', 'NYG', 'NYJ', 'MIA', 
         'IND', 'DET', 'GB', 'TB', 'SF', 'CHI', 'PHI', 'ARI', 'DEN', 'KC', 'SF', 'MIN', 'PHI', 'NE', 'ATL', 
         'TEN', 'OAK', 'BUF', 'OAK', 'GB', 'MIA', 'CHI', 'OAK', 'CLE', 'PHI', 'MIA', 'DAL', 'LAC', 'IND', 'ATL', 
         'BUF', 'CLE', 'ATL', 'CLE', 'NYJ', 'HOU', 'BUF', 'PIT', 'IND', 'ARI', 'LAC', 'CAR', 'PIT', 'DEN', 'NO', 
         'MIN', 'OAK', 'LAC', 'CLE', 'DAL', 'LAC', 'LAC', 'IND', 'PIT', 'NYG', 'PIT', 'BUF', 'NYG', 'NE', 'OAK', 
         'JAC', 'BAL', 'JAC', 'CIN', 'CHI', 'DEN', 'CAR', 'NYJ', 'IND', 'JAC', 'LA', 'CAR', 'NO', 'OAK', 'NO', 
         'NYJ', 'CAR', 'LA', 'DEN', 'NE', 'CLE']
}

In [3]:
df = pd.DataFrame(data)
df.head()

,full_name,rec_yards,team
0,Michael Thomas,1688,NO
1,Chris Godwin,1333,TB
2,DeAndre Hopkins,1165,HOU
3,Julio Jones,1316,ATL
4,Cooper Kupp,1062,LA


## Objective

Make a new dataframe containing only the top four entries per team by highest rec_yards

## Method 1 - make a count of the items per group then select the top 4

In [4]:
# Make a sorted multi-index
df_sorted = df.set_index(['team','rec_yards']).sort_index(ascending=False)

# Add a dummy column containing all 1s
df_sorted['count'] = 1

# Turn it into a ranking by team
df_sorted['count'] = df_sorted.groupby('team')['count'].cumsum()       

# Only keep the first n rows per team
n = 4
df_sorted = df_sorted.loc[df_sorted['count'] <= n]

# Re-arrange it however you prefer
df_sorted.reset_index().set_index(['team','count'])

rec_yards         full_name
team count                             
WAS  1            919    Terry McLaurin
     2            332     Kelvin Harmon
     3            245   Paul Richardson
     4            198        Trey Quinn
TEN  1            927        A.J. Brown
...               ...               ...
ATL  4            155      Justin Hardy
ARI  1            759  Larry Fitzgerald
     2            649    Christian Kirk
     3            285      Damiere Byrd
     4            219     Pharoh Cooper

[127 rows x 2 columns]

## Method 2 - Sort first, then use head with groupby

In [5]:
n = 4
df.sort_values(['team','rec_yards'], ascending=False).groupby('team').head(4)

,full_name,rec_yards,team
21,Terry McLaurin,919,WAS
81,Kelvin Harmon,332,WAS
76,Paul Richardson,245,WAS
93,Trey Quinn,198,WAS
20,A.J. Brown,927,TEN
...,...,...,...
117,Justin Hardy,155,ATL
27,Larry Fitzgerald,759,ARI
34,Christian Kirk,649,ARI
89,Damiere Byrd,285,ARI


## Method 3 - using `DataFrame.nlargest`

In [6]:
df.groupby('team').apply(pd.DataFrame.nlargest, 4, columns=['rec_yards'])

full_name  rec_yards team
team                                      
ARI  27   Larry Fitzgerald        759  ARI
     34     Christian Kirk        649  ARI
     89       Damiere Byrd        285  ARI
     92      Pharoh Cooper        219  ARI
ATL  3         Julio Jones       1316  ATL
...                    ...        ...  ...
TEN  118     Kalif Raymond        170  TEN
WAS  21     Terry McLaurin        919  WAS
     81      Kelvin Harmon        332  WAS
     76    Paul Richardson        245  WAS
     93         Trey Quinn        198  WAS

[127 rows x 3 columns]

## Method 4 - using `pd.SeriesGroupBy.nlargest`

In [7]:
top_n_by_team = df.groupby('team')['rec_yards'].nlargest(4)  # Only works for one series
top_n_by_team

team     
ARI   27      759
      34      649
      89      285
      92      219
ATL   3      1316
             ... 
TEN   118     170
WAS   21      919
      81      332
      76      245
      93      198
Name: rec_yards, Length: 127, dtype: int64

In [8]:
df.loc[top_n_by_team.index.levels[1].values].set_index('team','full+name').sort_index(ascending=False)

,full_name,rec_yards
team,,
WAS,Terry McLaurin,919
WAS,Paul Richardson,245
WAS,Kelvin Harmon,332
WAS,Trey Quinn,198
TEN,A.J. Brown,927
...,...,...
ATL,Calvin Ridley,866
ARI,Larry Fitzgerald,759
ARI,Damiere Byrd,285


## Speed tests

In [9]:
def method1():

    # Make a sorted multi-index
    df_sorted = df.set_index(['team','rec_yards']).sort_index(ascending=False)

    # Add a dummy column containing all 1s
    df_sorted['count'] = 1

    # Turn it into a ranking by team
    df_sorted['count'] = df_sorted.groupby('team')['count'].cumsum()       

    # Only keep the first 4 rows per team
    df_sorted = df_sorted.loc[df_sorted['count'] <= 4]

def method2():
    top4_by_team = df.sort_values(['team','rec_yards'], ascending=False).groupby('team').head(4)
    
def method3():
    df.groupby('team').apply(pd.DataFrame.nlargest, 4, columns=['rec_yards'])
    
def method4():
    top4_by_team = df.groupby('team')['rec_yards'].nlargest(4).index.levels[1].values
    top4_by_team = df.loc[top4_by_team].set_index('team','full+name').sort_index(ascending=False)

In [10]:
%timeit method1()

6.34 ms ± 757 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%timeit method2()

3.02 ms ± 216 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%timeit method3()

62 ms ± 816 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%timeit method4()

21.8 ms ± 754 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
